In [ ]:
%matplotlib inline
from matplotlib import pylab as plt

### Make sure you have the geoserver VM running

this should show something like:

```
Current machine states:

geoserver                 running (virtualbox)

The VM is running. To stop this VM, you can run `vagrant halt` to
shut it down forcefully, or you can run `vagrant suspend` to simply
suspend the virtual machine. In either case, to restart it again,
simply run `vagrant up`.
```

You can do this by going to the geoserver/ directory and running ```vagrant up```

In [ ]:
!cd ../geoserver && vagrant status

### Display geoserver status

This should ensure the client can successfully connect to your VM,  if you do not see the Geoserver 'Status' page then something is wrong!

In [ ]:
from IPython.core.display import display, HTML
from geonotebook.config import Config
geoserver = Config().vis_server
display(HTML(geoserver.c.get("/about/status").text))

### Get the rgb.tif file from data.kitware.com

In [ ]:
!wget -O /tmp/rgb.tif https://data.kitware.com/api/v1/file/57bf01ff8d777f10f269cff4/download

### Add the layer to the map

In [ ]:
## Proposed API moving forward ##

# gt = GeoTiff("/tmp/rgb.tif")
# gt.describe()

# M.add_layer(gt.bands([0, 1, 2],  range=[(0, 1), (0, 1), (0, 1)], opacity=0.5))

M.add_layer("/tmp/rgb.tif")

In [ ]:
M.layers['rgb'].region

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(12, 12))

# Display BRG
# n = M.layers['rgb'].region

# Flip the second axis to get RGB
n = np.apply_along_axis(np.flipud, 2, M.layers['rgb'].region)
ax.imshow(n, interpolation='none')

In [ ]:
from scipy import ndimage
fig, ax = plt.subplots(figsize=(12, 12))

k = np.array([[[1, 1, 1], [1, 1, 1], [1, 1, 1]],
              [[1, 1, 1], [1, 1, 1], [0, 0, 0]],
              [[1, 1, 1], [0, 0, 0], [0, 0, 0]]])

ax.imshow(ndimage.convolve(n, k, mode='constant', cval=0.0), interpolation='none')

In [ ]:
M.remove_layer('rgb')

In [ ]:
M.layers